In [1]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import function
from tensorflow.python.framework import ops
from tensorflow.python.framework import test_util
from tensorflow.python.ops import gradients

In [2]:
graph = tf.Graph()

In [3]:
sess = tf.InteractiveSession(graph=graph)

In [4]:
class Test(object):
  def __init__(self, session):
    self.sess = session
    self.w_g  = tf.Variable([0.5], name="w_g")
    self.b_g  = tf.Variable([1.0], name="b_g")
    self.w1_f = tf.constant([1.0])
    self.w2_f = tf.constant([2.0])
    self.b_f  = tf.constant([1.0])
    self.n    = tf.constant([3.0])
    self.lr   = tf.constant([0.01])
    self.__name__ = "Test"
    
  @classmethod
  def policy(cls, x, w_g, b_g):
    return w_g * x * x + b_g

  @classmethod
  def value(cls, x, y, w1_f, w2_f, b_f):
    return w1_f * x + w2_f * y + b_f

  def weights_update(self):
    return [self.w_g.assign_add(self.lr * self.grads_p[0]), self.b_g.assign_add(self.lr * self.grads_p[1])]

  def testGrads(self):
    g = function.Defun(x=tf.float32, w_g=tf.float32, b_g=tf.float32)(self.policy)
    f = function.Defun(x=tf.float32, y=tf.float32, w1_f=tf.float32, w2_f=tf.float32, b_f=tf.float32)(self.value)
    x = tf.constant([2.0, 3.0, 1.0], name="x")
    y = g(x, tf.convert_to_tensor(self.w_g), tf.convert_to_tensor(self.b_g))
    v = f(x, y, self.w1_f, self.w2_f, self.b_f)

    # Build gradient graph (should add SymbolicGradient node for function).
    grads_v = gradients.gradients(v, [x, y])
    action_v_grad = grads_v[1]
    self.grads_p = gradients.gradients(y, [self.w_g, self.b_g], grad_ys=action_v_grad)
    for i, pg in enumerate(self.grads_p):
            self.grads_p[i] = pg / self.n
            
    update = self.weights_update()            
    
    print self.sess.run(v)
    print '---------------'
    print self.sess.run(grads_v)
    print '---------------'
    print self.sess.run(self.grads_p)
    print '---------------'
    print self.sess.run(update)
    print '---------------'
    print self.sess.run(v)
    
    # print self.sess.run(grads_weighted)[0]
    # print self.sess.run(grads_weighted)[1]

In [5]:
with graph.as_default():
  t = Test(sess)
  sess.run(tf.initialize_all_variables())
  t.testGrads()
  print 'xxxxxxxxxxxxxxxxxxxxx'
  t.testGrads()
  print 'xxxxxxxxxxxxxxxxxxxxx'
  t.testGrads()
  print 'xxxxxxxxxxxxxxxxxxxxx'
  t.testGrads()
  print 'xxxxxxxxxxxxxxxxxxxxx'
  t.testGrads()
  print 'xxxxxxxxxxxxxxxxxxxxx'
  t.testGrads()

[  9.  15.   5.]
---------------
[array([ 5.,  7.,  3.], dtype=float32), array([ 2.,  2.,  2.], dtype=float32)]
---------------
[array([ 9.33333302], dtype=float32), array([ 2.], dtype=float32)]
---------------
[array([ 0.5933333], dtype=float32), array([ 1.01999998], dtype=float32)]
---------------
[  9.78666687  16.72000122   5.22666645]
xxxxxxxxxxxxxxxxxxxxx
[  9.78666687  16.72000122   5.22666645]
---------------
[array([ 5.74666643,  8.11999989,  3.37333322], dtype=float32), array([ 2.,  2.,  2.], dtype=float32)]
---------------
[array([ 9.33333302], dtype=float32), array([ 2.], dtype=float32)]
---------------
[array([ 0.68666661], dtype=float32), array([ 1.03999996], dtype=float32)]
---------------
[ 10.57333279  18.43999863   5.4533329 ]
xxxxxxxxxxxxxxxxxxxxx
[ 10.57333279  18.43999863   5.4533329 ]
---------------
[array([ 6.49333286,  9.23999977,  3.74666643], dtype=float32), array([ 2.,  2.,  2.], dtype=float32)]
---------------
[array([ 9.33333302], dtype=float32), array([ 2

In [ ]:
sess.close()